# RNN Models

### Imports

In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM

### Load data

In [2]:
covid = pd.read_csv("../datasets/WHO-COVID-19-global-data.csv")

### Create mini datasets of each country with the first day having >= 100 cases as the start date

In [3]:
covid_shifted = covid[covid['Cumulative_cases'] >= 100]

In [4]:
ls = []
for country in covid_shifted['Country'].unique():
# a dataframe of an individual country
# starting at the first day with over 100 cases
# with just the date and cumulative cases
# with the index reset
    temp_df = covid_shifted[(covid_shifted['Cumulative_cases'] >= 100) 
                            & (covid_shifted['Country'] == country)][['Date_reported', 'Cumulative_cases']].reset_index().drop(columns='index')
    temp_df['Days_since_100'] = [i for i in range(len(temp_df))]
# add it to the list as a tuple to keep country name while reducing df size
    ls.append((country, temp_df))

In [5]:
# view the first three rows of the first four country reports
for tup in ls[0:2]:
    print(tup[0])
    display(tup[1])

Afghanistan


,Date_reported,Cumulative_cases,Days_since_100
0,2020-03-28,106,0
1,2020-03-29,114,1
2,2020-03-30,114,2
3,2020-03-31,166,3
4,2020-04-01,192,4
...,...,...,...
461,2021-07-02,122156,461
462,2021-07-03,123485,462
463,2021-07-04,124748,463
464,2021-07-05,125937,464


Albania


,Date_reported,Cumulative_cases,Days_since_100
0,2020-03-25,108,0
1,2020-03-26,136,1
2,2020-03-27,148,2
3,2020-03-28,159,3
4,2020-03-29,174,4
...,...,...,...
463,2021-07-01,132521,463
464,2021-07-02,132523,464
465,2021-07-03,132526,465
466,2021-07-04,132534,466


## Simple RNN

In [10]:
# test country
country = ls[1]

step = 4

X = [val for val in country[1]['Cumulative_cases']]
# add "step" extra to be used to make the final pred
X = np.append(X, X[-1::]*step)

In [11]:
# convert into dataset matrix
# credit to https://www.datatechnotes.com/2018/12/rnn-example-with-keras-simplernn-in.html
def convertToMatrix(data, step):
 X, Y =[], []
 for i in range(len(data)-step):
  d=i+step  
  X.append(data[i:d,])
  Y.append(data[d,])
 return np.array(X), np.array(Y)

In [12]:
X_train, y_train =convertToMatrix(X,step)

In [13]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

In [14]:
# SimpleRNN model
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(1,step), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 32)                1184      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,729
Trainable params: 1,729
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X_train, y_train, epochs=244, batch_size=16)

Epoch 1/244
30/30 [==============================] - 2s 3ms/step - loss: 2089966208.0000
Epoch 2/244
30/30 [==============================] - 0s 2ms/step - loss: 26253360.0000
Epoch 3/244
30/30 [==============================] - 0s 2ms/step - loss: 667129.6250
Epoch 4/244
30/30 [==============================] - 0s 1ms/step - loss: 1628580.0000
Epoch 5/244
30/30 [==============================] - 0s 2ms/step - loss: 3071014.5000
Epoch 6/244
30/30 [==============================] - 0s 2ms/step - loss: 1675662.8750
Epoch 7/244
30/30 [==============================] - 0s 2ms/step - loss: 2429592.2500
Epoch 8/244
30/30 [==============================] - 0s 2ms/step - loss: 2010366.7500
Epoch 9/244
30/30 [==============================] - 0s 2ms/step - loss: 2103569.0000
Epoch 10/244
30/30 [==============================] - 0s 2ms/step - loss: 1959149.2500
Epoch 11/244
30/30 [==============================] - 0s 2ms/step - loss: 1971066.7500
Epoch 12/244
30/30 [=============================

In [ ]:
preds = model.predict(X_train)

In [ ]:
df = pd.DataFrame(preds)

In [ ]:
df.size

In [ ]:
df['orig'] = X[0:468]

In [ ]:
df[0] = [int(i) for i in df[0]]

In [ ]:
df

---
# Automation
### Function and loop

In [7]:
# convert into dataset matrix
# credit to https://www.datatechnotes.com/2018/12/rnn-example-with-keras-simplernn-in.html
def convertToMatrix(data, step):
 X, Y =[], []
 for i in range(len(data)-step):
  d=i+step  
  X.append(data[i:d,])
  Y.append(data[d,])
 return np.array(X), np.array(Y)

In [20]:
# accepts a tuple (country name, dataset)
def make_model(country_tuple):
    country = country_tuple[1]
    
    step = 4
    X = [val for val in country['Cumulative_cases']]
#     add "step" extra to be used to make the final pred
    X = np.append(X, X[-1::]*step)
    
    X_train, y_train =convertToMatrix(X,step)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    
#     SimpleRNN model
    model = Sequential()
    model.add(SimpleRNN(units=32, input_shape=(1,step), activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='rmsprop')
    model.summary()
    
    model.fit(X_train, y_train, epochs=240, batch_size=16)

    #TODO: return a tuple with the country and the model
    return country_tuple, model

In [ ]:
# loop to create a list of models
model_ls = []

for country in ls:
    model_ls.append(make_model(country))
    

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 32)                1184      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,729
Trainable params: 1,729
Non-trainable params: 0
_________________________________________________________________
Epoch 1/240
30/30 [==============================] - 2s 2ms/step - loss: 221742848.0000
Epoch 2/240
30/30 [==============================] - 0s 2ms/step - loss: 930101.7500
Epoch 3/240
30/30 [==============================] - 0s 3ms/step - loss: 1906345.8750
Epoch 4/240
30/30 [==============================] - 0s 2ms/step - loss: 2176080.2500
Epoch 5/240
30/30 [==================

In [ ]:
# loop to make a predictive grade for each country sorted descending in score

for model in model_ls:
    

---

## LSTM

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(32))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()